In [2]:
import os
import h5py

In [4]:
def compare_hdf5_structure(file1, file2):
    """
    Compare the structure of two HDF5 files.
    
    Parameters:
        file1 (str): Path to the first HDF5 file.
        file2 (str): Path to the second HDF5 file.
    
    Returns:
        bool: True if structures are identical, False otherwise.
    """
    def recurse_compare(group1, group2, path='/'):
        if sorted(group1.keys()) != sorted(group2.keys()):
            print(f"Structure mismatch at {path}: Different children.")
            return False
        for name in group1:
            if type(group1[name]) != type(group2[name]):
                print(f"Structure mismatch at {path}{name}: Different types.")
                return False
            if isinstance(group1[name], h5py.Dataset):
                if group1[name].shape != group2[name].shape or group1[name].dtype != group2[name].dtype:
                    print(f"Structure mismatch at {path}{name}: Different shape or dtype.")
                    return False
            elif isinstance(group1[name], h5py.Group):
                if not recurse_compare(group1[name], group2[name], path + name + '/'):
                    return False
        return True

    with h5py.File(file1, 'r') as f1, h5py.File(file2, 'r') as f2:
        return recurse_compare(f1, f2)

In [6]:
# Check the structure of the two files
map_foldername = '2d-quad'
map_folder = os.path.join('/Users/liujunyu/Desktop/Research/UVic_NYU/IGA_IPC/code/polyfem/experiments/mesh/iga-ipc', map_foldername)
map_filename = '2d-quad-map.hdf5'
map_filepath = os.path.join(map_folder, map_filename)

reference_map_filename = 'triangle_ref1-P2.hdf5'
reference_map_folder = '/Users/liujunyu/Desktop/Research/UVic_NYU/IGA_IPC/code/polyfem/experiments/high-order-ipc-data/weights/higher_order'
reference_map_filepath = os.path.join(reference_map_folder, reference_map_filename)

files_same_structure = compare_hdf5_structure(map_filepath, reference_map_filepath)
print("The files have the same structure:" if files_same_structure else "The files do not have the same structure.")

Structure mismatch at /ordered_edges: Different shape or dtype.
The files do not have the same structure.


In [8]:
import h5py

def compare_hdf5_structure_and_attributes(file1, file2):
    """
    Compare the structure and attributes of two HDF5 files.
    
    Parameters:
        file1 (str): Path to the first HDF5 file.
        file2 (str): Path to the second HDF5 file.
    
    Returns:
        bool: True if structures and attributes are identical, False otherwise.
    """
    def recurse_compare(group1, group2, path='/'):
        if sorted(group1.keys()) != sorted(group2.keys()):
            print(f"Structure mismatch at {path}: Different children.")
            return False
        for name in group1:
            if type(group1[name]) != type(group2[name]):
                print(f"Type mismatch at {path}{name}.")
                return False
            # Check attributes
            attrs1, attrs2 = group1[name].attrs.items(), group2[name].attrs.items()
            if sorted(attrs1) != sorted(attrs2):
                print(f"Attribute mismatch at {path}{name}: {attrs1} != {attrs2}")
                return False
            if isinstance(group1[name], h5py.Dataset):
                if group1[name].dtype != group2[name].dtype:
                    print(f"Dtype mismatch at {path}{name}.")
                    return False
            elif isinstance(group1[name], h5py.Group):
                if not recurse_compare(group1[name], group2[name], path + name + '/'):
                    return False
        return True

    with h5py.File(file1, 'r') as f1, h5py.File(file2, 'r') as f2:
        return recurse_compare(f1, f2)

In [9]:
# Check the structure of the two files
files_same_structure = compare_hdf5_structure_and_attributes(map_filepath, reference_map_filepath)
print("The files have the same structure and attributes:" if files_same_structure else "The files do not have the same structure and attributes.")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [10]:
import h5py

def print_attributes(filename):
    with h5py.File(filename, 'r') as file:
        def recursive_print(name, node):
            print(f"Attributes of {name}: {dict(node.attrs)}")
            if isinstance(node, h5py.Group):
                for child_name, child_node in node.items():
                    recursive_print(f"{name}/{child_name}", child_node)

        recursive_print('/', file)

print_attributes(map_filepath)
print_attributes(reference_map_filepath)

Attributes of /: {}
Attributes of //ordered_edges: {}
Attributes of //ordered_faces: {}
Attributes of //weight_triplets: {'shape': array([4, 4])}
Attributes of //weight_triplets/cols: {}
Attributes of //weight_triplets/rows: {}
Attributes of //weight_triplets/values: {}
Attributes of /: {}
Attributes of //ordered_edges: {}
Attributes of //ordered_faces: {}
Attributes of //weight_triplets: {'shape': array([54, 15])}
Attributes of //weight_triplets/cols: {}
Attributes of //weight_triplets/rows: {}
Attributes of //weight_triplets/values: {}


In [7]:
import h5py
import numpy as np

def adjust_hdf5_to_match(file_to_adjust, reference_file):
    with h5py.File(reference_file, 'r') as ref, h5py.File(file_to_adjust, 'r+') as adj:
        # Iterate through all items in the reference file and adjust the target file accordingly
        def adjust_group(ref_group, adj_group):
            # Adjust datasets
            for name, ref_dataset in ref_group.items():
                if isinstance(ref_dataset, h5py.Dataset):
                    # Read reference data and dtype
                    ref_data = ref_dataset[()]
                    ref_dtype = ref_dataset.dtype

                    # If dataset exists in adj, modify it, otherwise create it
                    if name in adj_group:
                        adj_group[name][()] = ref_data.astype(ref_dtype)
                    else:
                        adj_group.create_dataset(name, data=ref_data, dtype=ref_dtype)

                    # Adjust attributes to match
                    for attr_name, attr_value in ref_dataset.attrs.items():
                        adj_group[name].attrs[attr_name] = np.array(attr_value, dtype=type(attr_value))

                elif isinstance(ref_dataset, h5py.Group):
                    # Recursively adjust groups
                    if name not in adj_group:
                        adj_group.create_group(name)
                    adjust_group(ref_dataset, adj_group[name])

        adjust_group(ref, adj)

# Adjust '2d-quad-map.hdf5' to match 'triangle_ref1-P2.hdf5'
adjust_hdf5_to_match('2d-quad-map.hdf5', 'triangle_ref1-P2.hdf5')

TypeError: Can't broadcast (9, 2) -> (261, 2)